In [ ]:
import pandas as pd

# Load Data

In [ ]:
df = pd.read_csv('train_with_30.csv')
df['date'] = pd.to_datetime(df['date'])
df['day_of_year'] = df['date'].dt.dayofyear
df['day_of_month'] = df['date'].dt.day

print(df)

# Merge Weather

Should contain columns date, weather_rain, weather_temp

In [ ]:
weather_df = pd.read_csv('your_weather_data.csv')
df = df.merge(weather_df[['weather_rain','weather_temp']], on='date',how='left')
print(df)

# Create dataframe in right format

You need to have a dataframe that already includes columns date, visitors, bookingCount_{days_before}, weather_temp and weather_rain.

In [ ]:
def expand_df(original_df):
    expanded_rows = []
    for _, row in original_df.iterrows():
        for days_before in [4, 7, 10, 21,]:
            weather_temp = row['weather_temp']
            weather_rain = row['weather_rain']
            weather_available = 1
            if days_before in [21,10]:
                weather_temp = 0
                weather_rain = 0
                weather_available = 0
            new_row = {
                'date': row['date'],
                'days_before': row['days_before'],
                'bookings': row['bookings'],
                'weather_temp': weather_temp,
                'weather_rain': weather_rain,
                'weather_available': weather_available,
                'visitors': row['visitors'],
            }
            expanded_rows.append(new_row)
    return pd.DataFrame(expanded_rows)
new_df = expand_df(df)
print(new_df)

# Merge dataframe with statistical increases

In [ ]:
# Se till så att du inte inkluderar de datum som du tänkt använda som test i modell träningen här, för du vill ha statistiskt från datan innan
df_statistical_training = new_df[new_df['date'].between('2022-01-01', '2023-12-30')].copy()
print(df_statistical_training)

In [ ]:
df_statistical_training['bookings_increase'] = df_statistical_training.apply(lambda row: ((row['visitors'] - row['bookings']) / row['bookings']) if row['bookings'] > 0 else 0, axis=1)
df_statistical_training = df_statistical_training.groupby(
    ['day_of_year', 'day_of_month', 'days_before']
)['bookings_increase'].median().reset_index()

if set(['day_of_year', 'day_of_month', 'days_before']).issubset(new_df.columns) and \
   set(['day_of_year', 'day_of_month', 'days_before', 'bookings_increase']).issubset(df_statistical_training.columns):
    final_df = new_df.merge(
        df_statistical_training[['day_of_year', 'day_of_month', 'days_before', 'bookings_increase']],
        on=['day_of_year', 'day_of_month', 'days_before'],
        how='left'
    )
    print(final_df)
else:
    print("Nödvändiga kolumner för sammanslagning saknas i en eller båda DataFrames!")
print(final_df)

# Uncomment the row below to save new dataframe

In [ ]:
# final_df.to_csv('train_data.csv')